## Setup the Google Embedding 

In [77]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [78]:
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

In [79]:
from google import genai
client = genai.Client(api_key = GEMINI_API_KEY)

In [ ]:
input_text = "The world could be a better place with but human fails."
google_embedding = client.models.embed_content(
        model="gemini-embedding-001",
        contents=input_text)

print(google_embedding.embeddings)

In [84]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
langchain_google_embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001", google_api_key=GEMINI_API_KEY)

# embeddings.embed_query(input_text)

## Get data via ingestion

In [ ]:
import json 
import httpx 


json_data = httpx.get("https://api.artic.edu/api/v1/artworks").json()
json_data

In [72]:
from langchain_text_splitters import RecursiveJsonSplitter
json_splitter = RecursiveJsonSplitter(max_chunk_size=300)
json_chunks = json_splitter.split_json(json_data)
docs = json_splitter.create_documents(texts=[json_data])

In [73]:
docs

[Document(metadata={}, page_content='{"pagination": {"total": 129884, "limit": 12, "offset": 0, "total_pages": 10824, "current_page": 1, "next_url": "https://api.artic.edu/api/v1/artworks?page=2"}}'),
 Document(metadata={}, page_content='{"data": [{"id": 22, "api_model": "artworks", "api_link": "https://api.artic.edu/api/v1/artworks/22", "is_boosted": false, "title": "Villa Pamphili outside Porta S. Pancrazio, from Views of Rome", "alt_titles": null, "thumbnail": {"lqip": "", "width": 4373, "height": 3164, "alt_text": "A work made of etching on heavy ivory laid paper."}, "main_reference_number": "1887.255", "has_not_been_viewed_much": false, "boost_rank": null, "date_start": 1776, "date_end": 1776, "date_display": "1776, published 1800\\u201307", "date_qualifier_title": "Made", "date_qualifier_id": 4, "artist_display": "Giovanni Battista Piranesi (Italian, 1720-1778)\\npublished by Francesco (Italian, 1758-1810) and Pietro Piranesi (Italian, born 1758/9)", "place_of_origin": "Italy", "

## Using vector storeDB

In [80]:
from langchain_chroma import Chroma

In-Memory: Vector stored temporarily

In [85]:
vector_store = Chroma(
    collection_name="test-in-memory",
    embedding_function=langchain_google_embeddings
)

with Data Persistence: Vector stored in a folder or directory

In [86]:
vector_store = Chroma(
    collection_name="test-with-persistence",
    embedding_function=langchain_google_embeddings,
    persist_directory="./chroma_langchain_db"
)

In [87]:
vector_store.add_documents(docs)

['1506d8a4-687b-47fa-8d8b-8d8671168277',
 '51e4b550-3bdf-4dbb-8b7a-ba1e32ba9090',
 'e3feca94-04ac-414b-8f22-4b3c8c64b265',
 '7cdcfe92-23af-4966-9128-beeb6e97dc0e']

In [92]:
input_text = """
"id": 11723
"""
similarity_search_result = vector_store.similarity_search(
    input_text,
    k=2
)

for res in similarity_search_result:
    print(f"{res.page_content}\n")

{"data": [{"id": 22, "api_model": "artworks", "api_link": "https://api.artic.edu/api/v1/artworks/22", "is_boosted": false, "title": "Villa Pamphili outside Porta S. Pancrazio, from Views of Rome", "alt_titles": null, "thumbnail": {"lqip": "", "width": 4373, "height": 3164, "alt_text": "A work made of etching on heavy ivory laid paper."}, "main_reference_number": "1887.255", "has_not_been_viewed_much": false, "boost_rank": null, "date_start": 1776, "date_end": 1776, "date_display": "1776, published 1800\u201307", "date_qualifier_title": "Made", "date_qualifier_id": 4, "artist_display": "Giovanni Battista Piranesi (Italian, 1720-1778)\npublished by Francesco (Italian, 1758-1810) and Pietro Piranesi (Italian, born 1758/9)", "place_of_origin": "Italy", "description": null, "short_description": null, "dimensions": "Image: 49 \u00d7 70.3 cm (19 5/16 \u00d7 27 11/16 in.); Plate: 49.5 \u00d7 71 cm (19 1/2 \u00d7 28 in.); Sheet: 51.6 \u00d7 71.8 cm (20 3/8 \u00d7 28 5/16 in.)", "dimensions_deta